# import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ECz
from tqdm import tqdm
import time

from bs4 import BeautifulSoup as bs

# URL 가져와

In [3]:
# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()

# 제품 목록 페이지 로드
url = 'https://www.lg.com/in/tv-soundbars/all-tv-soundbars/?ec_model_status_code=ACTIVE&sortCriteria=%40ec_spotlights_order_no+descending%2C%40ec_salable_status+descending%2C%40ec_pdp_view_count+descending'
driver.get(url)

# 페이지 로딩 대기
time.sleep(3)

# 정적 스크랩핑으로 raw 웹 데이터 저장
soup = bs(driver.page_source, 'lxml')
links = soup.select('ul div.c-product-item__ufn a')

# 모든 제품 페이지에서 제품 링크 수집
product_links = []

for link in links:
    # product_links.append('https://www.lg.com/'+link['href'])
    product_links.append('https://www.lg.com'+link['href'])
product_links

['https://www.lg.com/in/tv-soundbars/qned/65qned90t6a/',
 'https://www.lg.com/in/tv-soundbars/4k-uhd-tvs/55ur7500psc/',
 'https://www.lg.com/in/tv-soundbars/oled/oled55g46la/',
 'https://www.lg.com/in/tv-soundbars/oled/oled48c46la/',
 'https://www.lg.com/in/tv-soundbars/qned/75qned82t6a/',
 'https://www.lg.com/in/tv-soundbars/qned/75qned88t6a/',
 'https://www.lg.com/in/tv-soundbars/smart-tvs/32lq643bpta/',
 'https://www.lg.com/in/tv-soundbars/smart-tvs/32lm563bptc/',
 'https://www.lg.com/in/tv-soundbars/oled-evo/oled55c3psa/',
 'https://www.lg.com/in/tv-soundbars-1/4k-uhd-tvs/55uq7550psf/',
 'https://www.lg.com/in/tv-soundbars/smart-tvs/43lm5600ptc/',
 'https://www.lg.com/in/tv-soundbars/tv-accessories/akb76043109/']

# 방법1: 요소 하나하나 가져오기

In [38]:
# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()

# for i in tqdm(range(len(product_links))):
for i in product_links[1:2]:
    
    # driver.get(product_links[i])
    driver.get(i)
    
    # 리뷰 탭 클릭
    review_tab = driver.find_elements(By.CLASS_NAME, 'c-tabs__item-text')
    review_tab[3].click()
    time.sleep(5)  # 리뷰 탭 로딩 대기

    # Shadow DOM의 root 접근 함수 선언
    def shadow_root(element):
        shadow_root = driver.execute_script('return arguments[0].shadowRoot',element)
        return shadow_root

    # 리뷰 데이터가 있는 섹션 찾기
    shadow_data = driver.find_element(By.CSS_SELECTOR, '[data-bv-show="reviews"]')
    shadow_data_open = shadow_root(shadow_data)

    
    # 스크롤 내려서 Locale 필터 설정
    element = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl')
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)  # 스크롤 완료 대기
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)  # 스크롤 완료 대기

    # Local filter
    locale_filter = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl > div > div:nth-child(4) > div')
    time.sleep(2)
    locale_filter.click()
    time.sleep(1)
    locale_filter.click()
    english_india = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-1tvt4yi-5.kveQCZ')
    english_india[1].click()
    time.sleep(1)

    # Lowest to Highest Rating
    low_to_high = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-11r39gb-1.ewPPY > div > div.bv-rnr__wnep33-0.cIbVOf > div > div.bv-rnr__sc-13ijx62-0.bmAjQc > div > div')
    time.sleep(2)
    low_to_high.click()
    time.sleep(1)
    low_to_high.click()
    low_to_high_select = shadow_data_open.find_element(By.ID, 'bv-reviews-filters-negative-3')
    low_to_high_select.click()
    time.sleep(1)
    
    # 리뷰 점수 가져오기
    scores = shadow_data_open.find_element(By.ID, 'reviews_container')
    scores = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewRating"]')
    score_list = []
    for score in scores:
        score_list.append(score.text)

    # 리뷰 본문 가져오기
    scores = shadow_data_open.find_element(By.ID, 'reviews_container')
    scores = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewBody"]')
    score_list = []
    for score in scores:
        score_list.append(score.text)

    # 리뷰 점수 가져오기
    scores = shadow_data_open.find_element(By.ID, 'reviews_container')
    scores = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewRating"]')
    score_list = []
    for score in scores:
        score_list.append(score.text)

    # 리뷰 점수 가져오기
    scores = shadow_data_open.find_element(By.ID, 'reviews_container')
    scores = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewRating"]')
    score_list = []
    for score in scores:
        score_list.append(score.text)

    # 리뷰 점수 가져오기
    scores = shadow_data_open.find_element(By.ID, 'reviews_container')
    scores = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewRating"]')
    score_list = []
    for score in scores:
        score_list.append(score.text)

    # 리뷰 점수 가져오기
    scores = shadow_data_open.find_element(By.ID, 'reviews_container')
    scores = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewRating"]')
    score_list = []
    for score in scores:
        score_list.append(score.text)

# 방법2: 다가져와서 전처리 해

In [9]:
# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()

# for i in tqdm(range(len(product_links))):
for i in product_links[2:3]:
    
    driver.get(i)
    
    # 리뷰 탭 클릭
    review_tab = driver.find_elements(By.CLASS_NAME, 'c-tabs__item-text')
    review_tab[3].click()
    time.sleep(3)  # 리뷰 탭 로딩 대기

    # Shadow DOM의 root 접근 함수 선언
    def shadow_root(element):
        shadow_root = driver.execute_script('return arguments[0].shadowRoot',element)
        return shadow_root
    
    # 리뷰 데이터가 있는 섹션 찾기
    shadow_data = driver.find_element(By.CSS_SELECTOR, '[data-bv-show="reviews"]')
    shadow_data_open = shadow_root(shadow_data)

    # 스크롤 내려서 Locale 필터 설정
    #bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl > div
    #bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl
    element = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl')
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)  # 스크롤 완료 대기
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)  # 스크롤 완료 대기

    # Local filter
    locale_filter = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl > div > div:nth-child(4) > div')
    time.sleep(2)
    locale_filter.click()
    time.sleep(1)
    locale_filter.click()
    english_india = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-1tvt4yi-5.kveQCZ')
    english_india[1].click()
    time.sleep(1)

    # Lowest to Highest Rating
    low_to_high = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-11r39gb-1.ewPPY > div > div.bv-rnr__wnep33-0.cIbVOf > div > div.bv-rnr__sc-13ijx62-0.bmAjQc > div > div')
    time.sleep(2)
    low_to_high.click()
    time.sleep(1)
    low_to_high.click()
    low_to_high_select = shadow_data_open.find_element(By.ID, 'bv-reviews-filters-negative-3')
    low_to_high_select.click()
    time.sleep(1)

    # 리뷰 컨테이너 1
    users = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__emkap-0.eyhvsJ')
    user_list = []
    for user in users:
        user_list.append(user.text)

    # 리뷰 컨테이너 2
    review_a_list = []
    try:
        reviews_a = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.eIsLQk')
        for review_a in reviews_a:
            review_a_list.append(review_a.text)
    except:
        pass
    
    # 리뷰 컨테이너 2
    review_b_list = []
    try:
        reviews_b = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.hDttlK')
        for review_b in reviews_b:
            review_b_list.append(review_b.text)
    except:
        pass
    review_a_b_list = review_a_list + review_b_list

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (548, -183)
  (Session info: chrome=128.0.6613.86)
Stacktrace:
	GetHandleVerifier [0x00007FF639DCB632+29090]
	(No symbol) [0x00007FF639D3E6E9]
	(No symbol) [0x00007FF639BFB1CA]
	(No symbol) [0x00007FF639C56C0E]
	(No symbol) [0x00007FF639C54682]
	(No symbol) [0x00007FF639C51B9B]
	(No symbol) [0x00007FF639C50DB5]
	(No symbol) [0x00007FF639C42D61]
	(No symbol) [0x00007FF639C766EA]
	(No symbol) [0x00007FF639C426A6]
	(No symbol) [0x00007FF639C76900]
	(No symbol) [0x00007FF639C965D9]
	(No symbol) [0x00007FF639C76493]
	(No symbol) [0x00007FF639C409B1]
	(No symbol) [0x00007FF639C41B11]
	GetHandleVerifier [0x00007FF63A0E883D+3294125]
	GetHandleVerifier [0x00007FF63A134423+3604371]
	GetHandleVerifier [0x00007FF63A12A2E7+3563095]
	GetHandleVerifier [0x00007FF639E86F16+797318]
	(No symbol) [0x00007FF639D4986F]
	(No symbol) [0x00007FF639D45454]
	(No symbol) [0x00007FF639D455E0]
	(No symbol) [0x00

In [58]:
review_a_list

["1 out of 5 stars.\nSTAY AWAY FROM THESE TVS\nA DAY AGO\nNever buy LG LED tv. They use cheap parts which will have issues within some months. I had purchased 2 times and both times tv didn't last more than 2 years. Just now I have 43' LED tv from LG which has panel issue but LG is not able to resolve the issue. Stay away from LG tvs.\nHow long have you owned this product?Over a year\nWhat is your primary reason for choosing LG?Online Research\nWas this an in-store or online purchase?In-store\nSelect the user type that you most relate toTech Lover\nNo, I do not recommend this product.\nOriginally posted on 43UR7500PSC\nHelpful?\n0 Helpful(0)\n0 Not helpful(0)\nReport",
 '1 out of 5 stars.\nPROBLEM IN PANEL WITHIN A WEEK OF PURCHASE\n10 MONTHS AGO\nI have purchased TV on 07/10/2023 and it shows problem in panel with in a week. I would not recommend product for others\nHow long have you owned this product?A few days\nWhat is your primary reason for choosing LG?Product Quality\nWas this a

-----------------

In [13]:
# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()

driver.get(product_links[2])

# 리뷰 탭 클릭
review_tab = driver.find_elements(By.CLASS_NAME, 'c-tabs__item-text')
review_tab[3].click()
time.sleep(3)  # 리뷰 탭 로딩 대기

# Shadow DOM의 root 접근 함수 선언
def shadow_root(element):
    shadow_root = driver.execute_script('return arguments[0].shadowRoot',element)
    return shadow_root

# 리뷰 데이터가 있는 섹션 찾기
shadow_data = driver.find_element(By.CSS_SELECTOR, '[data-bv-show="reviews"]')
shadow_data_open = shadow_root(shadow_data)

# 스크롤 내려서 Locale 필터 설정
element = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl > div > div:nth-child(4) > div')
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
time.sleep(2)  # 스크롤 완료 대기

제목 따로해! / 지역/성별/아이디 따로해!

In [40]:
# name 가져오기
names = shadow_data_open.find_element(By.ID, 'reviews_container')
names = names.find_elements(By.CSS_SELECTOR, '[class="bv-rnr__sc-1r4hv38-0 dQRRFq"]')
name_list = []
for name in names:
    name_list.append(name.text)
name_list

['gags', 'Sushil', 'DSC', 'Abhimanyu', 'Satish', 'Gee', 'Madhu0117']

In [36]:
# 리뷰 점수 가져오기
scores = shadow_data_open.find_element(By.ID, 'reviews_container')
scores = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewRating"]')
score_list = []
for score in scores:
    score_list.append(score.text)
score_list

['1 out of 5 stars.',
 '1 out of 5 stars.',
 '1 out of 5 stars.',
 '3 out of 5 stars.',
 '4 out of 5 stars.',
 '4 out of 5 stars.',
 '4 out of 5 stars.',
 '4 out of 5 stars.']

In [15]:
# 리뷰 컨테이너1 가져오기
a = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__emkap-0.eyhvsJ')
# a = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewRating"]')
a_list = []
for b in a:
    a_list.append(b.text)
a_list

['Manish',
 'G max\nMangalore\nGenderMale\nAge35 to 44',
 'Jay\nPandharapur MH\nGenderMale\nAge35 to 44',
 'EmeraldWall\nHaldwani\nGenderMale\nAge25 to 34',
 'Surag\nTrivandrum\nGenderMale\nAge45 to 54',
 'Mitesh\nSurat\nGenderMale\nAge18 to 24',
 'Saurabh\nJaipur\nGenderMale\nAge25 to 34',
 'Sravan\nGenderMale\nAge25 to 34',
 'Tomm Dinakara',
 'janu',
 'Ramesh T\nGenderMale\nAge35 to 44',
 'Thiru123\nIndia hyderabad\nGenderMale\nAge25 to 34',
 'Japras\nJakarta\nGenderMale\nAge25 to 34',
 'Anonymous\nHapur, U.P. India 245101\nGenderMale\nAge45 to 54',
 'Mohan\nVellore\nGenderMale\nAge25 to 34',
 'Manish\nGodda, jharkhand\nGenderMale\nAge25 to 34',
 'Balaji\nChennai\nGenderMale\nAge25 to 34',
 'Roshan\nTamil Nadu\nGenderMale\nAge25 to 34',
 'Swapnil lingayat\nRatnagiri\nGenderMale',
 'Buddy\nFaridabad\nGenderMale\nAge45 to 54',
 'ctn\nbangalore\nGenderMale\nAge35 to 44']

In [14]:
# 리뷰 컨테이너2 가져오기
a = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.eIsLQk')
# a = scores.find_elements(By.CSS_SELECTOR, '[itemprop="reviewRating"]')
a_list = []
for b in a:
    a_list.append(b.text)
a_list

['5 out of 5 stars.\nBEST TV ALL WORLD\n7 DAYS AGO\nThis is best tv I have experience in the television\nHow long have you owned this product?A few months\nWhat is your primary reason for choosing LG?Product Quality\nWas this an in-store or online purchase?In-store\nOriginally posted on OLED65G46LA\nHelpful?\n0 Helpful(0)\n0 Not helpful(0)\nReport',
 '5 out of 5 stars.\n20 DAYS AGO\nOne of best in oled With Dolby Atmos sound Dolby Vision\nBest FeaturesEasy To Use, Quality, Style/Design, Performance, Features/Settings\nHow long have you owned this product?A few days\nWhat is your primary reason for choosing LG?Product Quality\nWas this an in-store or online purchase?In-store\nSelect the user type that you most relate toTech Lover\nOriginally posted on OLED65G46LA\nHelpful?\n0 Helpful(0)\n0 Not helpful(0)\nReport',
 "5 out of 5 stars.\nGREAT REFRESH RATE AND PICTURE\nA MONTH AGO\nI bought this tv 65g4 just because of 144hz refresh rate... because my OLED samsung 65s95d refresh rate was n

In [ ]:
# 리뷰 제목 가져오기
titles = shadow_data_open.find_elements(By.CSS_SELECTOR, '.bv-content-title')
for title in titles:
    print(f"Title: {title.text}")

# 리뷰 본문 가져오기
bodies = shadow_data_open.find_elements(By.CSS_SELECTOR, '.bv-content-summary-body-text')
for body in bodies:
    print(f"Body: {body.text}")

# 리뷰 작성자 성별 가져오기
genders = shadow_data_open.find_elements(By.CSS_SELECTOR, '.bv-author-profile-gender')
for gender in genders:
    print(f"Gender: {gender.text}")

# 리뷰 작성자 위치 가져오기
locations = shadow_data_open.find_elements(By.CSS_SELECTOR, '.bv-author-profile-location')
for location in locations:
    print(f"Location: {location.text}")

In [ ]:
# 브라우저 종료
driver.quit()

In [ ]:
# 리뷰 데이터 수집
reviews = shadow_data_open.find_element(By.ID, 'reviews_container')
reviews = reviews.find_elements(By.CSS_SELECTOR, '[itemprop="reviewBody"]')
review_list = []
for review in reviews:
    review_list.append(review.text)
review_list

In [ ]:
# Locale 필터 열기

locale_filter = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl > div > div:nth-child(4) > div')
time.sleep(2)
locale_filter.click()
time.sleep(1)
locale_filter.click()

english_india = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-1tvt4yi-5.kveQCZ')
english_india[1].click()
time.sleep(1)

In [ ]:
# Lowest to Highest Rating
 
low_to_high = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-11r39gb-1.ewPPY > div > div.bv-rnr__wnep33-0.cIbVOf > div > div.bv-rnr__sc-13ijx62-0.bmAjQc > div > div')
time.sleep(2)
low_to_high.click()
time.sleep(1)
low_to_high.click()
 
low_to_high_select = shadow_data_open.find_element(By.ID, 'bv-reviews-filters-negative-3')
low_to_high_select.click()
time.sleep(1)